In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp email

# Compose and send emails

> Compose and send html emails through an SMTP server using TLS.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import smtplib
from email.message import EmailMessage
import mimetypes
from pathlib2 import Path
import re

## Complose a message

In [ ]:
#export
def create_html_message(from_address, to_addresses, subject, html, text = None, image_path = Path.cwd()):
    msg = EmailMessage()
    msg['From'] = from_address
    msg['To'] = to_addresses
    msg['Subject'] = subject
    if text is not None:
        msg.set_content(text)
    msg.add_alternative(html, subtype='html')
    
    cid_images = list(re.findall(fr'<img src="cid:(.*?)"', html))
    cid_images.extend(list(re.findall(fr'url\(cid:(.*?)\)', html)))
    cid_images = list(set(cid_images))
    for cid_img in cid_images:
        with open(image_path / cid_img, 'rb') as img:
            msg.get_payload()[-1].add_related(img.read(),'image', 'jpeg', cid = cid_img)
    return msg

### Add an attachment to a message

In [ ]:
#export
def add_attachmet(msg, path):
    "Add an attachment with location `path` to the cunnet message `msg`."
    # Guess the content type based on the file's extension.  Encoding
    # will be ignored, although we should check for simple things like
    # gzip'd or compressed files.
    ctype, encoding = mimetypes.guess_type(path)
    if ctype is None or encoding is not None:
        # No guess could be made, or the file is encoded (compressed), so
        # use a generic bag-of-bits type.
        ctype = 'application/octet-stream'
    maintype, subtype = ctype.split('/', 1)
    with open(path, 'rb') as f:
        msg.add_attachment(f.read(), maintype = maintype, subtype = subtype, filename = path.name)
    return msg

## Send a message using SMTP and TLS

In [ ]:
#export
def send_smtp_email(server, tls_port, user, pw, msg):
    "Send the message `msg` using the specified `server` and `port` - login using `user` and `pw`."
    # Create a secure SSL context
    try:
        smtp = smtplib.SMTP(server, tls_port)
        smtp.starttls()
        smtp.login(user, pw)
        smtp.send_message(msg)
    except Exception as e:
        print(e)
    finally:
        smtp.quit()

## Examples

The following is an example on how to compose and send a html-email message.

In [ ]:
## set user and password of the smtp server
#user = ''
#pw = ''
#
## send email from and to myself
#from_email = user
#to_email = ''
#
#html = """
#Hello, this is a test message!
#<h1>Hello 22!</h1>
#<img src="cid:email.jpg">
#<h1>Hello 23!</h1>
#<img src="cid:iceberg.jpg">
#"""
#
#msg = create_html_message(from_email, to_email, 'subject', html, image_path=Path(''))
#add_attachmet(msg, Path(''))
#
## uncomment after setting user and pw above
##send_smtp_email('', 587, user, pw, msg)